# Interacting with Thoth

Deployed Thoth, as of now, offers one API service for users. This endpoint exposes all the actions that can be done with Thoth, how one can interact with Thoth, how to consume data, use cluster for computation by supplying work and so. The API service exposes OpenAPI/Swagger documentation in a form of JSON so one can use Swagger user UIs, testing and tooling aroung OpenAPI/Swagger available out there.

The Swagger documentation is available under `/api/<version>/swagger.json` endpoint. In this Jupyter notebook one can see how to interact with the cluster, how to check responses and how to get cluster or job statuses. Let's start with defining Thoth's user API service where our Thoth deployment sits. If you access Thoth's user API service, you will get Swagger UI that will give you a documentation of the current user API service.

Now let's obtain actual service host so that the host is not exposed in the Jupyter notebook (it's an internal URL). We will use `obtain_location()` function to obtain URL to actual Thoth deployment:

In [1]:
from thoth.lab import obtain_location

THOTH_USER_API = obtain_location('thoth-test-upshift', verify=False)

In this Jupyter notebook we will use a library [requests](http://docs.python-requests.org/en/master/) to interact with endpoints directly. However, you can also use libraries such as [bravado](http://bravado.readthedocs.io/en/latest) that is a client for API endpoints defined by OpenAPI/Swagger specification.

Let's instantiate the client object that encapsulates all the logic for us:

In [2]:
import requests
from urllib.parse import urljoin

client = requests.get(urljoin(THOTH_USER_API, '/api/v1/swagger.json'))

Thoth user API exposes models that can be used to interact with the API. Let's get some models and let's check what do they offer.

In [3]:
Log = client.json().get('definitions').get('Log')
Packages = client.json().get('definitions').get('Packages')

Besides inherited attributes and methods, the `Packages` model also provides requirement property that, based on the docstring, is a string that represents stack requirements as they would be stated in the `requirements.txt` file.

You can browse intaractive pop-up via calling (shown in the pannel bellow in the Jupyter notebook UI):

In [4]:
Packages?

Now, let's try to instantiate such model:

In [5]:
packages = {'requirements':'tensorflow==1.6.0'}
headers = {'content-type': 'application/json'}
payload = {
    'solver': 'fridex/thoth-solver-fc27', # Solver image to be used.
    'debug': True, # We can optionally turn on debug mode to keep track of actions that are done by solver image.
    'transitive':False # To Inspect all the transitive dependencies, set as True.
} 

And call the remote endpoint called `solve` that will try to find all packages that can be installed with the given package (based on requirements stated in the `tensorflow` package in version 1.6.0 and all the transitive dependencies):

In [6]:
import json


response = requests.post(
    urljoin(THOTH_USER_API, '/api/v1/solve'),
    json=packages,
    params=payload,
    headers=headers
)

print("Status Code: ",response.status_code)
# Our instantiated Package model holding requirements as stated in requirements.txt file.

Status Code:  202


Now let's check the response from the user API service. If everything went well, the remote API should response with 202 HTTP status code:

In [7]:
assert response.status_code == 202, "The HTTP status code was not 202 but {:d}".format(response.status_code)

And the response itself should hold all the relevant information such as parameters supplied (also the ones that were not explicitly supplied) and `pod_id` that is ID of the pod in which the solver image is run:

In [8]:
from pprint import pprint

pprint(response.json())
pod_id = response.json()['pod_id']

{'parameters': {'cpu_request': None,
                'debug': True,
                'memory_request': None,
                'packages': 'tensorflow==1.6.0',
                'solver': 'fridex/thoth-solver-fc27',
                'transitive': False},
 'pod_id': 'fridex-thoth-solver-fc27-thoth-solver-fc27-7jb5z'}


Now let's try to check status of the pod:

In [9]:
response = requests.get(urljoin(THOTH_USER_API, '/api/v1/status/'+pod_id))
pprint(response.json())

{'pod_id': 'fridex-thoth-solver-fc27-thoth-solver-fc27-7jb5z',
 'status': {'waiting': {'reason': 'ContainerCreating'}}}


Thoth-core scheduled a pod in thoth-middleend namespace (a restricted environment). Let's wait for container creation to finish:

In [10]:
from time import sleep

while 'waiting' in response.json()['status']:
    print("Waiting for solver to start analyzing packages...")
    sleep(2)
    response = requests.get(urljoin(THOTH_USER_API, '/api/v1/status/'+pod_id))

Waiting for solver to start analyzing packages...
Waiting for solver to start analyzing packages...


As of now, the solver started analyzing dependencies:

In [11]:
pprint(response.json())

{'pod_id': 'fridex-thoth-solver-fc27-thoth-solver-fc27-7jb5z',
 'status': {'running': {'startedAt': '2018-06-12T17:54:06Z'}}}


As we started the solver in verbose mode, we can check what is going on right now:

In [12]:
sleep(20)
response = requests.get(urljoin(THOTH_USER_API, '/api/v1/log/'+pod_id))

Now let's print last ten lines of the pod log:

In [13]:
response.json()['pod_log'].split('\n')[-10:]

['INFO:thoth:Logging to rsyslog endpoint is turned off',
 'DEBUG:thoth.solver.cli:Debug mode is on',
 '']

Once the analysis finishes, all the results are stored onto [Ceph](https://ceph.com/), the distributed storage system. You can retrieve raw results by calling the following endpoint:

In [14]:
while True:
    response = requests.get(urljoin(THOTH_USER_API, '/api/v1/status/'+pod_id))
    if 'running' not in response.json()['status']:
        break

    print("Waiting for solver to finish analyzing packages...")
    sleep(10)
response = requests.get(urljoin(THOTH_USER_API, '/api/v1/solve/'+pod_id))

Waiting for solver to finish analyzing packages...
Waiting for solver to finish analyzing packages...
Waiting for solver to finish analyzing packages...


And let's display result that is stored onto Ceph distrubuted filesystem:

In [15]:
pprint(response.json())

{'metadata': {'analyzer': 'thoth-solver',
              'analyzer_version': '1.0.2',
              'arguments': {'pypi': {'exclude_packages': None,
                                     'index': None,
                                     'no_pretty': False,
                                     'no_transitive': True,
                                     'output': 'http://result-api/api/v1/solver-result',
                                     'requirements': 'tensorflow==1.6.0'},
                            'thoth-solver': {'verbose': True}},
              'datetime': '2018-06-12T17:54:59.604139',
              'distribution': {'codename': 'Twenty Seven',
                               'id': 'fedora',
                               'like': '',
                               'version': '27',
                               'version_parts': {'build_number': '',
                                                 'major': '27',
                                                 'minor': ''}},
     

## Solver images

As can be seen above, there are multiple solver images. They differ mainly in the base image. In the example shown in this Jupyter notebook, we run a solver that is based on Fedora:27 base image.

Requested solver is trying to install all the packages that you submitted in the `requirements` attribute via `Packages` model respecting version range specification. Thoth's solver tries to solve the given version range against index (PyPI index by default) and checks which versions satisfy the given version range specification at the solving time. That are also the very first observations that Thoth's solver stores about a package:

 > * Is the given package available?
 > * What versions of the given package satisfy the given version range specification at the analyzing time?

The next step of Thoth's solver is to actually try to install provided packages in resolved versions into the base image environment. With this step Thoth gathers additional observation:

 > * Can I install the given package into the base image environment?

If the given package is not installable, thoth-solver keeps track of errors that were encountered during installation for additional analyses. The reasons why a package is not installable can be numerous (such as expectation of some native binary such as `gcc` for installation that is not available in the given container environment, package not installable for the given Python version and so).

After each installation, the container environment is restored so that installation of a package does not affect solver environment for subsequent installations.

The steps discussed above are repeated for each and every package supplied via the `requirements` attribute in the `Packages` model and, if requested, for all the transitive dependencies of supplied packages. With this, Thoth also gathers addition observation:

 > * What are all the package requirements that satisfy package version ranges at the given point of time?

The result of a solver is a JSON document describing observations above and additional metadata such as Python interpreter used and its version available in the solver environment. These documents are subsequently additionally analysed and processed in the Thoth.